In [ ]:
%cd ../..

# Single flow - Multi Layer Perceptron classification of CIDDS-001 OpenStack data
The implementation is based on the paper **Intelligent Cyber Attack Detetion and Classification for Network-Based Intrusion Detection Systems**. However, the training data that was used is now balanced. The data of the first and second week are joined together to obtain the biggest possible dataset to train with.

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import pandas as pd
import numpy as np
from utils import cidds_001 as utils

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.2.0


## Load datasets

In [2]:
week1 = utils.load_internal_week1()
week2 = utils.load_internal_week2()

## Preprocess datasets
* Both datasets week1 and week2 are used to obtain as much malicous traffic of the underrepresented attack types as possible.
* Split the features and labels of the new dataset and encode the labels with one hot encoding.

In [3]:
week1_benign = week1.where(week1['attack_type'] == '---').dropna()
week1_portScan = week1.where(week1['attack_type'] == 'portScan').dropna()
week1_dos = week1.where(week1['attack_type'] == 'dos').dropna()
week1_pingScan = week1.where(week1['attack_type'] == 'pingScan').dropna()
week1_bruteForce = week1.where(week1['attack_type'] == 'bruteForce').dropna()

In [4]:
print(f'len(week1_benign) = {len(week1_benign)}')
print(f'len(week1_portScan) = {len(week1_portScan)}')
print(f'len(week1_dos) = {len(week1_dos)}')
print(f'len(week1_pingScan) = {len(week1_pingScan)}')
print(f'len(week1_bruteForce) = {len(week1_bruteForce)}')

len(week1_benign) = 7010897
len(week1_portScan) = 183511
len(week1_dos) = 1252127
len(week1_pingScan) = 3359
len(week1_bruteForce) = 1626


In [5]:
week2_benign = week2.where(week2['attack_type'] == '---').dropna()
week2_portScan = week2.where(week2['attack_type'] == 'portScan').dropna()
week2_dos = week2.where(week2['attack_type'] == 'dos').dropna()
week2_pingScan = week2.where(week2['attack_type'] == 'pingScan').dropna()
week2_bruteForce = week2.where(week2['attack_type'] == 'bruteForce').dropna()

In [6]:
print(f'len(week2_benign) = {len(week2_benign)}')
print(f'len(week2_portScan) = {len(week2_portScan)}')
print(f'len(week2_dos) = {len(week2_dos)}')
print(f'len(week2_pingScan) = {len(week2_pingScan)}')
print(f'len(week2_bruteForce) = {len(week2_bruteForce)}')

len(week2_benign) = 8515329
len(week2_portScan) = 82407
len(week2_dos) = 1706900
len(week2_pingScan) = 2731
len(week2_bruteForce) = 3366


In [7]:
dataset = pd.concat(
    [
        week1_pingScan, week2_pingScan, # use all pingScan flows of both wee1 and week2
        week1_bruteForce, week2_bruteForce, # use all bruteForce flows of both week1 and week2
        week1_benign.sample(frac=1, random_state=13).head(6000), # but use a small random sample of data of all other attack types
        week2_benign.sample(frac=1, random_state=13).head(6000),
        week1_dos.sample(frac=1, random_state=13).head(6000),
        week2_dos.sample(frac=1, random_state=13).head(6000),
        week1_portScan.sample(frac=1, random_state=13).head(6000),
        week2_portScan.sample(frac=1, random_state=13).head(6000),
    ]
).sample(frac=1, random_state=13)

In [8]:
print(f'len(dataset) = {len(dataset)}')

len(dataset) = 47082


In [9]:
data_x = dataset.drop(columns=utils.columns_to_drop + ['attack_type'])
data_y = pd.get_dummies(dataset['attack_type'])

## Split training and testing datasets

In [10]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=13)

## Noramalize features

In [11]:
_ = utils.robust_scaling_normalization(x_train, utils.columns_to_normalize, cidds_df_test=x_test)

## Create and compile the MLP model

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(16,)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

## Fit the MLP model

In [14]:
x = x_train.to_numpy()
y = y_train.to_numpy()

In [15]:
model.fit(x=x, y=y,
          batch_size=1024,
          epochs=50)

Epoch 1/50
37/37 [==============================] - 0s 4ms/step - loss: 4.0404 - accuracy: 0.5075
Epoch 2/50
37/37 [==============================] - 0s 4ms/step - loss: 1.7402 - accuracy: 0.6876
Epoch 3/50
37/37 [==============================] - 0s 4ms/step - loss: 1.3277 - accuracy: 0.7603
Epoch 4/50
37/37 [==============================] - 0s 4ms/step - loss: 0.9353 - accuracy: 0.7985
Epoch 5/50
37/37 [==============================] - 0s 4ms/step - loss: 0.7574 - accuracy: 0.8208
Epoch 6/50
37/37 [==============================] - 0s 4ms/step - loss: 0.9012 - accuracy: 0.8340
Epoch 7/50
37/37 [==============================] - 0s 4ms/step - loss: 0.6227 - accuracy: 0.8404
Epoch 8/50
37/37 [==============================] - 0s 4ms/step - loss: 0.5644 - accuracy: 0.8451
Epoch 9/50
37/37 [==============================] - 0s 4ms/step - loss: 0.5016 - accuracy: 0.8509
Epoch 10/50
37/37 [==============================] - 0s 4ms/step - loss: 0.4725 - accuracy: 0.8556
Epoch 11/50
37/37 [

## Test the fitted model with unseen data

In [16]:
x = x_test.to_numpy()
y = y_test.to_numpy()

In [17]:
test_loss, test_acc = model.evaluate(x,  y, verbose=2)

print('\nTest accuracy:', test_acc)

295/295 - 0s - loss: 0.1562 - accuracy: 0.9557

Test accuracy: 0.9557183980941772


## Create a confusion matrix for the predictions on the test data

In [18]:
pred_y = model.predict(x)

In [19]:
y_lbl = np.argmax(y, axis=1)
pred_y_lbl = np.argmax(pred_y, axis=1)

In [20]:
tf.math.confusion_matrix(labels=y_lbl, predictions=pred_y_lbl, num_classes=5)

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[2229,   93,    5,    0,    8],
       [  26,  936,    0,    1,   47],
       [   0,    0, 2359,    0,    0],
       [  48,   22,    0, 1174,    6],
       [  35,   38,    0,   88, 2302]])>

In [21]:
print(f'Columns of data_y (and confusion matrix): {data_y.columns}')

Columns of data_y (and confusion matrix): Index(['---', 'bruteForce', 'dos', 'pingScan', 'portScan'], dtype='object')
